In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az

In [ ]:
# Load and prepare data
df = pd.read_csv(r'/content/bren.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)




In [ ]:
df.head()

In [ ]:
# Proper resampling with Date preserved
monthly_df = df.resample('M', on='Date').last().copy()
monthly_df['LogReturn'] = np.log(monthly_df['Price'] / monthly_df['Price'].shift(1))
monthly_df.dropna(inplace=True)

# Ensure 'Date' becomes a column again
monthly_df.reset_index(inplace=True)



In [ ]:
monthly_df.head()

In [ ]:
# 🎯 Step 4: Prepare log return data for modeling
returns = monthly_df['LogReturn'].values
n = len(returns)
print(f"Number of monthly data points: {n}")

In [ ]:
returns_std = (returns - np.mean(returns)) / np.std(returns)

with pm.Model() as model:
    tau = pm.DiscreteUniform('tau', lower=0, upper=n-1)
    mu1 = pm.Normal('mu1', mu=0, sigma=1)
    mu2 = pm.Normal('mu2', mu=0, sigma=1)
    sigma = pm.HalfNormal('sigma', sigma=1)
    mu = pm.math.switch(tau >= np.arange(n), mu1, mu2)
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=returns_std)

    trace = pm.sample(
        4000, tune=2000,
        chains=4, cores=4,
        target_accept=0.95,
        return_inferencedata=True,
        random_seed=42
    )

In [ ]:
# Trace plots to check convergence
az.plot_trace(trace)
plt.tight_layout()
plt.show()

# Summary statistics
summary = az.summary(trace, hdi_prob=0.95)
print(summary)

In [ ]:
print(monthly_df.head())



In [ ]:
# Step 7: Extract most likely change point index and visualize

# Get tau samples from the posterior and compute the most likely index
tau_samples = trace.posterior['tau'].values.flatten()
most_likely_tau = int(np.median(tau_samples))

# Get the date corresponding to the change point index
change_date = monthly_df.loc[most_likely_tau, 'Date']

print(f"Detected change point at index {most_likely_tau}, which corresponds to {change_date}")

# Plot the log returns with the detected change point
plt.figure(figsize=(14, 5))
plt.plot(monthly_df['Date'], monthly_df['LogReturn'], label='Log Return', color='blue')
plt.axvline(change_date, color='red', linestyle='--', label=f'Change Point: {change_date.date()}')
plt.title('Detected Change Point in Monthly Brent Oil Log Returns')
plt.xlabel('Date')
plt.ylabel('Log Return')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
events = pd.read_csv(r'/content/event_data_extended.csv')  # Adjust path if needed

In [ ]:
events.head()

In [ ]:

events = events[['event_name', 'date', 'description']]  # Drop unwanted columns
events['date'] = pd.to_datetime(events['date'])  # Convert to datetime


In [ ]:
# Find events near the detected change point (±30 days)
nearby_events = events[
    (events['date'] >= change_date - pd.Timedelta(days=30)) &
    (events['date'] <= change_date + pd.Timedelta(days=30))
]

print("Events near detected change point:")
print(nearby_events)


In [ ]:
print("Change point date:", change_date)


In [ ]:
events.sort_values('date')


In [ ]:
nearby_events = events[
    (events['date'] >= change_date - pd.Timedelta(days=6000)) &
    (events['date'] <= change_date + pd.Timedelta(days=6000))
]

print("Events within ±6000 days:")
print(nearby_events)


In [ ]:
print("Detected change point date:", change_date)


In [ ]:
# Calculate mean returns before and after the detected change point
before_returns = monthly_df.loc[:most_likely_tau - 1, 'LogReturn']
after_returns = monthly_df.loc[most_likely_tau:, 'LogReturn']

mean_before = before_returns.mean()
mean_after = after_returns.mean()

# Compute percent change relative to the magnitude of mean before
percentage_change = 100 * (mean_after - mean_before) / abs(mean_before)

print(f"Mean monthly return before change point: {mean_before:.4%}")
print(f"Mean monthly return after change point: {mean_after:.4%}")
print(f"Percentage change in average return: {percentage_change:.2f}%")


In [ ]:
jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --clear-output --inplace change_point_model.ipynb
